# Application of Graphs on HGCAL data

## Augmenting the data for application into graphs

In [3]:
#Importing the required packages
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import uproot

In [14]:
# Reading the root file
file=uproot.open("ntuple_pi+_100GeV_100keve.root:AllLayers")
#file.keys()
#Extracting the data
hit_x=file['hit_x'].array(library='ak')
hit_y=file['hit_y'].array(library='ak')
hit_z=file['hit_z'].array(library='ak')
hit_l=file['hit_l'].array(library='ak')
hit_E=file['hit_E'].array(library='ak')

Here, we need to pass data for each event to create graphs for each event. This can be done very easily if we can store our data for each event into a dictionary.

In [40]:
events = []   # list of events, each event is a dict

num_events = len(hit_x)   # Number of events

for i in range(num_events):
    event = {
        "event_n":i,
        "hit_x": hit_x[i],
        "hit_y": hit_y[i],
        "hit_z": hit_z[i],
        "hit_E": hit_E[i],
        "hit_l": hit_l[i]
    }
    events.append(event)


In [94]:
#events

## A few tests

### Checking out the dictionary

In [95]:
#Checking for each event
events[0] # The data for the event-1

{'event_n': 0,
 'hit_x': <Array [0, 0, 0, 0, 0, ..., -3.88, 3.88, -3.88, 5.82] type='404 * float64'>,
 'hit_y': <Array [0, 0, 0, 0, 0, ..., -1.12, 3.36, -3.36, 1.12] type='404 * float64'>,
 'hit_z': <Array [13.9, 14.8, 16.8, 17.7, ..., 138, 138, 138, 145] type='404 * float64'>,
 'hit_E': <Array [9.53e-05, 8.14e-05, ..., 0.000768, 0.000217] type='404 * float64'>,
 'hit_l': <Array [1, 2, 3, 4, 5, 6, 6, ..., 37, 37, 38, 38, 38, 39] type='404 * int32'>}

In [42]:
events[0]['hit_x'] # The hit_x values for the first event

<Array [0, 0, 0, 0, 0, ..., -3.88, 3.88, -3.88, 5.82] type='404 * float64'>

### Testing the masking

In [48]:
#Testing how to implement the mask
hit_E_ev=events[0]["hit_E"]
hit_x_ev=events[0]["hit_x"]
mask=hit_E_ev>0
hit_x_ev[mask]

<Array [0, 0, 0, 0, 0, ..., -3.88, 3.88, -3.88, 5.82] type='404 * float64'>

### Testing the nearest neighbours

In [68]:
from sklearn.neighbors import NearestNeighbors
test_event=events[0]
test_x=test_event["hit_x"][mask]
test_y=test_event["hit_y"][mask]
test_z=test_event["hit_z"][mask]
X=np.transpose(np.vstack([test_x,test_y,test_z]))
#X=np.vstack([test_x,test_y,test_z])

In [75]:
nbrs_test = NearestNeighbors(n_neighbors=8, algorithm='ball_tree').fit(X)

distances_test, indices_test = nbrs_test.kneighbors(X)

indices_test

array([[  0,   1,   2, ...,   5,   6,   7],
       [  1,   0,   2, ...,   5,   6,   7],
       [  2,   3,   1, ...,   5,   7,   6],
       ...,
       [401, 403, 400, ..., 402, 386, 387],
       [402, 400, 387, ..., 397, 396, 394],
       [403, 401, 400, ..., 385, 386, 387]])

In [76]:
indices_test[:,0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [83]:
neighbors_test=indices_test[:,1:8+1]

In [89]:
len(X)

404

In [92]:
#for i in range(len(X)):
#    for j in neighbors_test[i]:
#        print(j)

In [84]:
#Build the edge list:
edges=[]
for i in range(len(X)):
    for j in neighbors_test[i]:
        edges.append([i,j])

In [120]:
edges_ar=np.array(edges)
edges_ar.shape

(2828, 2)

In [105]:
edge_indices=torch.tensor(edges,dtype=torch.long).t().contiguous()
#edge_indices=torch.tensor(edges,dtype=torch.long)

In [106]:
edge_indices

tensor([[  0,   0,   0,  ..., 403, 403, 403],
        [  1,   2,   3,  ..., 385, 386, 387]])

## Creating the event graphs

In [43]:
#Importing the required packages
from torch_geometric.data import Data
import torch

In [113]:
#Creating graphs for each event
from sklearn.neighbors import NearestNeighbors
def create_graph(event,k=8):
    # event: dictionary containing the data for 1 event
    #Extracting the data for the points where Energy value of hits is non zero
    mask=event["hit_E"]>0
    x=np.array(event["hit_x"])[mask]
    y=np.array(event["hit_y"])[mask]
    z=np.array(event["hit_z"])[mask]
    l=np.array(event["hit_l"])[mask]
    E=np.array(event["hit_E"])[mask]
    
    #Developing the node feature matrix
    node_features=torch.tensor(np.transpose(np.vstack([x,y,z,E])),dtype=torch.float)
    
    #Determining the Edges- using K-Nearest Neighbours
    #Coordinates for computing distance
    coords=np.transpose(np.vstack([x,y,z]))
    nbrs=NearestNeighbors(n_neighbors=k+1,algorithm='auto').fit(coords)
    distances,indices=nbrs.kneighbors(coords)
    #indices[:,0] is the node itself. So we remove it
    neighbors=indices[:,1:k+1]
    
    #Build the edge list:
    edges=[]
    for i in range(len(coords)):
        for j in neighbors[i]:
            edges.append([i,j])
    
    edge_index=torch.tensor(edges,dtype=torch.long)
    
    data=Data(x=node_features,edge_index=edge_index.t().contiguous())
    return data

In [116]:
#Testing for our test event
test_event=events[0]
graph=create_graph(test_event,k=8)

In [117]:
graph

Data(x=[404, 4], edge_index=[2, 3232])

In [122]:
#Creating the graphs
#graph=[]
graph=[create_graph(event,k=8) for event in events]